In [1]:
#Imports
import sys
sys.path.append('../../python/')
import NGC5533_functions as nf
import noordermeer as noord
import fitting_NGC5533 as fitting

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interactive, fixed, FloatSlider, HBox, Layout, Button, Label, Output, VBox

from IPython.display import display, clear_output
from IPython.display import Javascript

import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")  #ignore warnings

In [2]:
# Data
data = noord.data
data_total = noord.data_total
r_dat = noord.r_dat
v_dat = noord.v_dat
v_err0 = noord.v_err0
v_err1 = noord.v_err1


print(len(v_dat))

69


In [3]:
# Define components
def blackhole(r,M):
    return nf.bh_v(r,M,load=False)

def bulge(r,bpref):
    return bpref*nf.b_v(r,load=True)

def disk(r,dpref):
    return dpref*nf.d_thief(r)

def halo(r,rc,rho00):
    return nf.h_v(r,rc,rho00,load=False)

def gas(r,gpref):
    return gpref*nf.g_thief(r)

def totalcurve(r,M,bpref,dpref,rc,rho00,gpref):
    total = np.sqrt(blackhole(r,M)**2 
                    + bulge(r,bpref)**2 
                    + disk(r,dpref)**2
                    + halo(r,rc,rho00)**2
                    + gas(r,gpref)**2)
    return total

In [4]:
# Fitting parameters
best_M = fitting.f_M
best_bpref = fitting.f_c
best_dpref = fitting.f_pref
best_rc = fitting.f_rc
best_rho00 = fitting.f_hrho00
best_gpref = fitting.f_gpref


In [5]:
# Define plotting function
def f(M,bpref,dpref,rc,rho00,gpref):
    
    # Define r
    r = np.linspace(0.1,100,1000)
    
    # Plot
    plt.figure(figsize=(9,7))
    plt.xlim(0,100)
    plt.ylim(0,360)
    
    plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
    plt.plot(r,blackhole(r,M), label=("Black Hole"))
    plt.plot(r,bulge(r,bpref), label=("Bulge"))
    plt.plot(r,disk(r,dpref), label=("Disk"))
    plt.plot(r,halo(r,rc,rho00), label=("Halo"))
    plt.plot(r,gas(r,gpref), label=("Gas"))
    plt.plot(r,totalcurve(r,M,bpref,dpref,rc,rho00,gpref), label=("Total Curve"))
    
    plt.legend()
    plt.show()
    
    #Print values
    print("Black Hole Mass = {:.2e} solar masses.".format(M))
    print("Bulge Prefactor = {:.2f}.".format(bpref))
    print("Disk Prefactor = {:.2f}.".format(dpref))
    print("Gas Prefactor = {:.2f}.".format(gpref))    
    print("Halo Core Radius = {:.2f} kpc.".format(rc))
    print("Halo Surface Density = {:.2e}.".format(rho00))
    
    # Print chi squared and reduced chi squared
    # Residuals
    r = np.linspace(0.1,100,69)
    residuals = v_dat - totalcurve(r_dat,M,bpref,dpref,rc,rho00,gpref)
    # Determining errors
    errors = np.sqrt(v_err1**2 + fitting.v_i**2) #second term is inclination uncertainty
    # Chi squared
    chisquared = np.sum(residuals**2/errors**2)
    #chisquared = stats.chisquare(v_dat,totalcurve(r,M,bpref,dpref,rc,rho00,gpref))
    reducedchisquared = chisquared * (1/(len(r_dat)-6))
    print("Chi squared = {:.5f}".format(chisquared))
    print("Reduced Chi squared = {:.5f}.".format(reducedchisquared))


In [6]:


# Appearance
style = {'description_width': 'initial'}
layout = {'width':'600px'}

# Define slides
M = FloatSlider(min=0, max=5e9, step=1e8, value=best_M, description='Black Hole Mass [$M_{\odot}$]', readout_format='.2e', orientation='horizontal', style=style, layout=layout)
bpref = FloatSlider(min=0, max=5, step=0.1, value=best_bpref, description='Bulge Prefactor', readout_format='.2f', orientation='horizontal', style=style, layout=layout)
dpref = FloatSlider(min=0, max=5, step=0.1, value=best_dpref, description='Disk Prefactor', readout_format='.2f', orientation='horizontal', style=style, layout=layout)
#rc = FloatSlider(min=0, max=5, step=0.1, value=best_rc, description='Halo Core Radius [kpc]', readout_format='.2f', orientation='horizontal', style=style, layout=layout)
rc = fixed(best_rc)
rho00 = FloatSlider(min=0, max=1e9, step=1e7, value=best_rho00, description='Halo Surface Density [$M_{\odot} / pc^3$]', readout_format='.2e', orientation='horizontal', style=style, layout=layout)
gpref = FloatSlider(min=0, max=5, step=0.1, value=best_gpref, description='Gas Prefactor', readout_format='.2f', orientation='horizontal', style=style, layout=layout)

# Interactive widget
def interactive_plot(f):
    interact = interactive(f, M = M, 
                               bpref = bpref, 
                               dpref = dpref, 
                               rc = rc,
                               rho00 = rho00,
                               gpref = gpref,
                               continuous_update=False)
    return interact

# Button to revert back to Best Fit
button = Button(
    description="Best Fit",
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='check')
out = Output()

def on_button_clicked(_):
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    M.value = best_M
    bpref.value = best_bpref
    dpref.value = best_dpref
    rho00.value = best_rho00
    gpref.value = best_gpref

button.on_click(on_button_clicked)

# displaying button and its output together
VBox([button,out,interactive_plot(f)])

#interactive_plot(f)
